In [6]:
import pandas as pd
import numpy as np
#import itertools
import sklearn.metrics as met
import xgboost as xgb
#from sklearn.ensemble import RandomForestClassifier
import scipy as sp

import sys
sys.path.insert(0, '../../notebooks/libs/')
import FeatureExtraction as FE

In [11]:
training = pd.read_csv('../../csv/chat_based_features_training.csv')
test = pd.read_csv('../../csv/chat_based_features_test.csv')

training_sparse = sp.sparse.csr_matrix(training.ix[:,1:].values, dtype=float)[:,:-1]
test_sparse = sp.sparse.csr_matrix(test.ix[:,1:].values, dtype=float)[:,:-1]

In [ ]:
training_xml = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_xml = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

matrix_training, matrix_testing = FE.make_tf_idf_matrix_from_XML(training_xml, test_xml)

In [ ]:
print matrix_training.shape
print training_sparse.shape

In [ ]:
training_all = sp.sparse.hstack((training_sparse, matrix_training))
test_all = sp.sparse.hstack((test_sparse, matrix_testing))

del matrix_training
del matrix_testing

In [12]:
#forest = RandomForestClassifier(max_depth=17, n_estimators=300, bootstrap=False, n_jobs=8)
#forest = forest.fit(training_all, np.ravel(training.iloc[:,[9]]))

sex_offender = xgb.XGBClassifier(max_depth=25, n_estimators=300, learning_rate=0.007, scale_pos_weight=5, gamma=7, objective='binary:logistic')
#sex_offender = xgb.XGBClassifier(max_depth=2, n_estimators=2)
sex_offender.fit(training_sparse, training.iloc[:,[9]])

#del training_all

/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=7, learning_rate=0.007, max_delta_step=0, max_depth=25,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=5, seed=0, silent=True, subsample=1)

In [13]:
#prediction = forest.predict(test_all)
prediction = sex_offender.predict(test_sparse)

count = 0
for p in prediction:
    if p == 1:
        count += 1
    
print count, len(prediction)

print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
print 'F1:', met.f1_score(test[['is sexual predator']], prediction)

239 218702
Accuracy:  0.99903521687
Precision:  0.589958158996
Recall: 0.555118110236
F1: 0.57200811359


In [ ]:
column_names = training.columns.values.tolist()[1:-1]
print column_names

In [ ]:
training.tail()

In [ ]:
# Remove scale_post_weight -> smaller F1 score, lower recall but increase precision
# Remove gama for bigger precision
sex_offender0 = xgb.XGBClassifier(max_depth=25, n_estimators=300, learning_rate=0.007, scale_pos_weight=5, gamma=7)
sex_offender1 = xgb.XGBClassifier(max_depth=25, n_estimators=300)
sex_offender2 = xgb.XGBClassifier(max_depth=5, n_estimators=220, learning_rate=0.007, scale_pos_weight=20, gamma=2)

In [ ]:
sex_offender0.fit(training[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']], training.iloc[:,[9]])
sex_offender1.fit(training[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']], training.iloc[:,[9]])
sex_offender2.fit(training[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']], training.iloc[:,[9]])

In [ ]:
prediction0 = sex_offender0.predict(test[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']])
prediction1 = sex_offender1.predict(test[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']])
prediction2 = sex_offender2.predict(test[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']])

In [ ]:
def bagging(vote_number, prediction_list):
    total_prediction = []
    for i in range(len(prediction_list[0])):
        voters = 0
        
        for prediction in prediction_list:
            if prediction[i] == 1:
                voters += 1
                
        if voters >= vote_number:
            total_prediction.append(1)
        else:
            total_prediction.append(0)
                
    return np.array(total_prediction)

prediction = bagging(2, [prediction0, prediction1, prediction2])

In [ ]:
print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
print 'F1:', met.fbeta_score(test[['is sexual predator']], prediction, 1)

In [ ]:
forest = RandomForestClassifier(n_estimators = 100)

max_f1 = 0
best_features = []

for num_features in range(2, len(column_names)-1):
    for column_name_subset in itertools.combinations(column_names, num_features):

        forest = forest.fit(training[list(column_name_subset)], np.ravel(training.iloc[:,[9]]))
        prediction = forest.predict(test[list(column_name_subset)])

        print column_name_subset
        print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
        print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
        print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
        print 'F1:', met.f1_score(test[['is sexual predator']], prediction)
        print "\n\n"

        f1 = met.f1_score(test[['is sexual predator']], prediction)
        if max_f1 < f1:
            max_f1 = f1
            best_features =  column_name_subset
                                                                                                 
print max_f1
print best_features